In [1]:
from garden_ai import GardenClient, Model, step
from garden_ai.mlmodel import LocalModel

In [2]:
# instantiate the Garden client
client = GardenClient()

In [3]:
# convert our saved model into a Garden LocalModel
with open("requirements.txt", "r") as f:
    pip_reqs = [line.strip() for line in f.readlines()]

local_model = LocalModel(model_name="dendrite_segmentation",
                         flavor="tensorflow",
                         extra_pip_requirements=pip_reqs,
                         local_path="model.h5",
                         user_email=client.get_email())

In [ ]:
# register the LocalModel with Garden using the client
registered_model = client.register_model(local_model)

In [5]:
# Garden's pipelines are composed of steps, so let's make one to run the model
@step
def run_inference(input_arg: object, model=Model(registered_model.model_uri)) -> object:
    return model.predict(input_arg)

In [ ]:
# add the step to a pipeline
pipeline = client.create_pipeline(["Isaac Darling"],
                                  "Dendrite Segmentation Test Pipeline",
                                  short_name=registered_model.model_name,
                                  steps=(run_inference,),
                                  requirements_file="requirements.txt",
                                  description="Semantic dendrite segmentation via machine learning",
                                  tags=["materials science",
                                        "x-ray",
                                        "segmentation",
                                        "computer vision"])

In [ ]:
# build a container designed specially to run the pipeline
container_uuid = client.build_container(pipeline)

In [10]:
# register the pipeline and its encompassing container to Garden!
func_uuid = client.register_pipeline(pipeline, container_uuid)
print(f"Created function with uuid: {func_uuid}!")

Created function with uuid: cf266298-49fe-48ac-b131-ac9259f8d220!


In [11]:
# create a garden and add the pipeline to it, then publish it to the search index
garden = client.create_garden(pipeline.authors,
                              pipeline.title,
                              pipelines=[pipeline],
                              description=pipeline.description,
                              tags=pipeline.tags)

client.publish_garden_metadata(garden)["state"]

'SUCCESS'

In [12]:
# confirm garden is searchable on the index
garden == client.get_published_garden(garden.uuid)

True